In [1]:
import ROOT
ROOT.TMVA.Tools.Instance()

Welcome to ROOTaaS 6.06/06


<ROOT.TMVA::Tools object at 0x5360860>

In [2]:
outfileName = "KNN_from_real_data_via_TMVAReg.root"
outputFile = ROOT.TFile(outfileName,"RECREATE")
factory = ROOT.TMVA.Factory("TMVARegression", outputFile,
                            "!V:!Silent:Color:DrawProgressBar" )

--- Factory                  : You are running ROOT Version: 6.06/06, Jul 6, 2016
--- Factory                  : 
--- Factory                  : _/_/_/_/_/ _|      _|  _|      _|    _|_|   
--- Factory                  :    _/      _|_|  _|_|  _|      _|  _|    _| 
--- Factory                  :   _/       _|  _|  _|  _|      _|  _|_|_|_| 
--- Factory                  :  _/        _|      _|    _|  _|    _|    _| 
--- Factory                  : _/         _|      _|      _|      _|    _| 
--- Factory                  : 
--- Factory                  : ___________TMVA Version 4.2.1, Feb 5, 2015
--- Factory                  : 


In [3]:
fname = "/root/Downloads/dataSet_CERN_to_RAL.root"
inputFile = ROOT.TFile.Open( fname )

In [4]:
regTree = inputFile.Get("TreeR")

In [5]:
leaves = regTree.GetListOfLeaves()
# add all variables, except throughput and timestamp
for i in xrange(len(leaves)):
    if (leaves[i].GetName() == "timestamp") or (leaves[i].GetName() == "throughput"):
            continue
    factory.AddVariable( leaves[i].GetName(), leaves[i].GetTitle(), "units", 'F' )
# Add the variable carrying the regression target
factory.AddTarget( "throughput" )

In [6]:
regWeight  = 1.0
factory.AddRegressionTree( regTree, regWeight )

# I don't know what to do about this.. guess we don't need it
# This would set individual event weights (the variables defined in the 
# expression need to exist in the original TTree)
#factory.SetWeightExpression( "var1", "Regression" )

numOfEvents = regTree.GetEntries()
percentageOfTrainingSample = 0.6
toTrain = int(numOfEvents*percentageOfTrainingSample)
# ToDo implement this with nTrainRegression=+str(toTrain)+:nTestRegression=0 in the options
# Apply additional cuts on the signal and background samples (can be different)
mycut = ROOT.TCut('') # for example: TCut mycut = "abs(var1)<0.5 && abs(var2-0.5)<1";
factory.PrepareTrainingAndTestTree( mycut, "nTrain_Regression="+str(toTrain)+":nTest_Regression=0:SplitMode=random:!V:NormMode=None" )

--- DataSetInfo              : Added class "Regression"	 with internal class number 0
--- Factory                  : Add Tree TreeR of type Regression with 639 events
--- DataSetInfo              : Class index : 0  name : Regression


In [24]:
# Book the actual method
# use the default for now
# but lets use three layers
factory.BookMethod( ROOT.TMVA.Types.kMLP, "MLP",
                   "!H:!V:VarTransform=None:NeuronType=tanh:NCycles=5000:HiddenLayers=N+4,N-5:LearningRate=0.005:TestRate=6:TrainingMethod=BFGS:Sampling=1:SamplingEpoch=1:ConvergenceImprove=1e-30:ConvergenceTests=10:UseRegulator" );

--- Factory                  : Booking method: MLP
--- MLP                      : Building Network
--- MLP                      : Initializing weights


In [ ]:
# Train MVAs using the set of training events
factory.TrainAllMethods();

# ---- Evaluate all MVAs using the set of test events
factory.TestAllMethods();

# ----- Evaluate and compare performance of all configured MVAs
factory.EvaluateAllMethods();  

In [ ]:
outputFile.Close()
print("==> Wrote root file: " + outputFile.GetName())
print("==> TMVAClassification is done!")
del factory
ROOT.TMVA.TMVAGui( outfileName )

In [ ]:
raw_input("Press Enter to continue...")